In [1]:
import cv2
import os
import numpy as np
import random

input_file = 'VDB\D.mp4'   

# Open Video

In [2]:
def open_vid(input_file):
    cap = cv2.VideoCapture(input_file)
    if not cap.isOpened():
        print("Error: Could not open video.")
    return cap

In [3]:
#cap = open_vid(input_file)

# Get Video Properties

In [4]:
"""width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("Width: ",width)
print("Height: ",height)
print("Original FPS: ",original_fps)
print("Frame Count: ",frame_count)"""

'width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))\nheight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))\noriginal_fps = cap.get(cv2.CAP_PROP_FPS)\nframe_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))\nprint("Width: ",width)\nprint("Height: ",height)\nprint("Original FPS: ",original_fps)\nprint("Frame Count: ",frame_count)'

In [5]:
def get_frames(cap):
    frames = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for i in range(frame_count):
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            return frames

        # Save the frame to the list
        frames.append(frame)
        
    if not frames:
        print("No frames were saved.")
    
    return frames

In [6]:
#cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
#Frames = get_frames(cap)

# Delete PNG Files in Directory

In [7]:
def delete_png_files(directory_path):
    # List all files in the specified directory
    for filename in os.listdir(directory_path):
        # Construct full file path
        file_path = os.path.join(directory_path, filename)
        try:
            # Check if it's a PNG file and remove it
            if os.path.isfile(file_path) and filename.lower().endswith('.png'):
                os.remove(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

# Read and Save Frames in Interval

In [8]:
def save_frames(frames, frame_folder):
    # Create the folder to save frames if it doesn't exist
    if not os.path.exists(frame_folder):
        os.makedirs(frame_folder)

    frame_count = 0
    for i in range(len(frames)):
        # Save the frame as an image file
        frame_filename = os.path.join(frame_folder, f'frame_{frame_count:03d}.png')
        cv2.imwrite(frame_filename, frames[i])
        frame_count += 1

    #if not frames:
         #print("No frames were saved.")

In [9]:
#delete_png_files('saved_frames')

In [10]:
#save_frames(Frames[50:100],"saved_frames")

# Create Video with Set of Frames

In [11]:
def save_vid(frames,output_file,fps):
    # Get frame dimensions
    height, width, _ = frames[0].shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

    # Write the frames to the new video file
    [out.write(frame) for frame in frames];

    # Release the video writer object
    out.release()

In [12]:
#save_vid(Frames[100:150], "slice_vid.mp4", original_fps)

# Change FPS

In [13]:
#save_vid(Frames[100:150],"slice_vid_new_fps.mp4",12)

# Play Frames

In [14]:
def play_frames(frames,fps):
    
    delay = int(1000/fps)
    print("Delay: ",delay)

    for frame in frames:
        # Display the frame
        cv2.imshow('Video Playback', frame)
        # Exit the playback if 'q' is pressed
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
            
    cv2.destroyAllWindows()

In [15]:
#play_frames(Frames[:100],24)

# Optical Flow

## Farneback Magnitude

In [16]:
def OFM(frames):
    
    OF = [] 
    #Convert to Grayscale
    prev_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)

    for i in range(len(frames)-1):
        #Convert the next frame to grayscale
        next_gray = cv2.cvtColor(frames[i+1], cv2.COLOR_BGR2GRAY)

        # Calculate the dense optical flow using Farneback method
        flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5,  3,  10,  10,    5,    1.2,   0)
                                            #prev      next       flow  dist lvl  win  it  smooth  std   flag 
 
        # Visualize the optical flow
        hsv = np.zeros_like(frames[i]) #[ch0,ch1,ch2]
        if len(hsv.shape) != 3 or  hsv.shape[2] != 3:
            hsv = np.zeros(frames[i].shape[0],frames[i].shape[1],3)
        hsv[..., 1] = 255 #ch1 Saturation (Full)

        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1]) #Cartesian to Polar
        hsv[..., 0] = ang * 180 / np.pi / 2  
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX) #Normalize from 0 to 255
        flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)   #Convert to RBG
        OF.append(flow_rgb)
        
        flow_M  = flow[...,0]+flow[...,1]
        
        # Display the original frame and the optical flow
        cv2.imshow('Prev Frame', frames[i])
        cv2.imshow('Next Frame', frames[i+1])
        cv2.imshow('Optical Flow', flow_rgb)
        cv2.imshow('Optical Flow Mag',flow_M)

        # Wait for a key press to move to the next frame
        if cv2.waitKey(0) & 0xFF==ord('q'):
            break

        # Update the previous frame and grayscale image
        prev_gray = next_gray

    # Release the video capture object and close all OpenCV windows
    cv2.destroyAllWindows()
    return OF

In [17]:
#Optical_R = OFM(Frames[100:150])

## Farneack with Vectors

In [18]:
def draw_optical_flow_vectors(flow, frame, step):
    h, w = frame.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    # Create a mask to draw the vectors
    mask = np.zeros_like(frame)
    
    # Create line endpoints
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    # Draw lines and circles
    for (x1, y1), (x2, y2) in lines:
        cv2.line(mask, (x1, y1), (x2, y2), (0, 255, 0), 1)
        cv2.circle(frame, (x1, y1), 1, (0, 255, 0), -1)

    return cv2.add(frame, mask)

def OFV(frames,step):
    OF = []
    
    #convert it to grayscale
    prev_gray = cv2.cvtColor(np.copy(frames[0]), cv2.COLOR_BGR2GRAY)

    for i in range(1,len(frames)):
        # next frame convert it to grayscale
        next_gray = cv2.cvtColor(np.copy(frames[i]), cv2.COLOR_BGR2GRAY)

        # Calculate the dense optical flow using Farneback method
        flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # Draw the optical flow vectors on the frame
        flow_frame = draw_optical_flow_vectors(flow, np.copy(frames[i]),step)
        OF.append(flow)
        
        # Display the original frame with optical flow vectors
        cv2.imshow('Prev Frame',frames[i-1])
        cv2.imshow('Next Frame',frames[i])
        cv2.imshow('Optical Flow Vectors', flow_frame)

        # Wait for a key press to move to the next frame
        if cv2.waitKey(0) & 0xFF == ord('q'):
            break

        # Update the previous frame and grayscale image
        prev_gray = next_gray

    # Release the video capture object and close all OpenCV windows
    cv2.destroyAllWindows()
    return OF

In [19]:
#Optical_V = OFV(Frames[100:150],20)

## Lucas-Kanade

In [20]:
def OFLK(frames,step_size):
    OF = []
    # Parameters for the Lucas-Kanade optical flow
    lk_params=dict(winSize=(step_size,step_size),maxLevel=3,criteria=(cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT,10, 0.03))

    # Take the first frame and convert it to grayscale
    old_frame = np.copy(frames[0])
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    # Create a grid of points to track
    grid_y, grid_x = np.mgrid[0:old_gray.shape[0]:step_size, 0:old_gray.shape[1]:step_size]
    p0 = np.vstack((grid_x.ravel(), grid_y.ravel())).T.astype(np.float32).reshape(-1, 1, 2)


    for i in range(1, len(frames)):
        frame = np.copy(frames[i])
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        # Select good points
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Draw the normalized vectors
        for j, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            # Calculate vector components
            dx = a - c
            dy = b - d
            # Normalize vector
            magnitude = np.sqrt(dx**2 + dy**2)
            if magnitude > 0:
                dx /= magnitude
                dy /= magnitude
            # Scale vector for visualization
            scale = 10  # Adjust this value for smaller or larger arrows
            a = int(c + dx * scale)
            b = int(d + dy * scale)
            frame = cv2.arrowedLine(frame, (int(c), int(d)), (a, b), (0, 255, 0), 1, tipLength=0.5)
            OF.append(frame)
        
        cv2.imshow('Prev',frames[i-1])
        cv2.imshow('Next',frames[i])
        cv2.imshow('Optical Flow Vectors', frame)
        
        if cv2.waitKey(0) & 0xff == ord('q'):
            break

        # Update the previous frame and previous points
        old_gray = frame_gray.copy()

    cv2.destroyAllWindows()
    return OF

In [21]:
#Optical_KL = OFLK(Frames[100:150],20)

## Phase Correlation

In [22]:
def PhaseC(frames, block_size=20, grid_step=20):
    OF = []

    for i in range(len(frames) - 1):
        prev_frame = frames[i]
        next_frame = frames[i + 1]

        # Convert frames to grayscale
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)

        # Create an image to visualize the flow
        flow_img = cv2.cvtColor(prev_gray, cv2.COLOR_GRAY2BGR)

        # Iterate over the grid
        for y in range(0, prev_gray.shape[0] - block_size, grid_step):
            for x in range(0, prev_gray.shape[1] - block_size, grid_step):
                # Extract the blocks
                prev_block = prev_gray[y:y + block_size, x:x + block_size]
                next_block = next_gray[y:y + block_size, x:x + block_size]

                # Compute phase correlation
                shift, _ = cv2.phaseCorrelate(prev_block.astype(np.float32), next_block.astype(np.float32))
                dx, dy = shift

                # Scale down the length of the arrows and size of the tips
                scale = 5
                tip_length = 0.2

                # Draw the vector on the flow image
                cv2.arrowedLine(flow_img, (x + block_size // 2, y + block_size // 2), 
                                (int(x + block_size // 2 + dx * scale), int(y + block_size // 2 + dy * scale)), 
                                (0, 255, 0), 1, tipLength=tip_length)

        OF.append(flow_img)

        # Display the frames and the flow
        cv2.imshow('Previous Frame', prev_frame)
        cv2.imshow('Next Frame', next_frame)
        cv2.imshow('Optical Flow', flow_img)

        # Wait for key press to proceed to the next frame
        if cv2.waitKey(0) & 0xff == ord('q'):
            break

    cv2.destroyAllWindows()
    return OF

In [23]:
#Optical_PC = PhaseC(Frames[100:150])

# Frames Differences

In [24]:
def frame_dif(frames, threshold=30):
    D = []
    prev_frame = frames[0]

    for i in range(1,len(frames)):

        # Convert frames to grayscale
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        curr_gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
        # Compute absolute difference between frames
        diff = cv2.absdiff(prev_gray, curr_gray)
              
        # Create a mask to highlight pixels with significant changes
        mask = np.zeros_like(diff)
        mask[diff > threshold] = 255
        D.append(mask)
        # Show original frame and mask
        cv2.imshow('Original Frame', frames[i])
        cv2.imshow('Pixels with Most Changes', mask)
        if cv2.waitKey(0) & 0xff == ord('q'):
            break

        prev_frame = frames[i]

    # Release the video capture object
    cv2.destroyAllWindows()
    return D

In [25]:
#Dif = frame_dif(Frames[200:250],20)

# Change Color Channel's Ranges

In [26]:
def change_range_colors(image, min_vals=(0, 0, 0), max_vals=(255, 255, 255)):
    # Split the image into its BGR channels
    b, g, r = cv2.split(image)
    
    # Clip each channel to its respective range
    b = np.clip(b, min_vals[0], max_vals[0])
    g = np.clip(g, min_vals[1], max_vals[1])
    r = np.clip(r, min_vals[2], max_vals[2])
    
    # Merge the channels back together
    new_image = cv2.merge((b, g, r))
    
    return new_image

In [27]:
""""Nframe = change_range_colors(Frames[100],(0,50,100),(250,200,250))
cv2.imshow('original',Frames[100])
cv2.imshow('changed',Nframe)
cv2.waitKey(0)
cv2.destroyAllWindows()"""

'"Nframe = change_range_colors(Frames[100],(0,50,100),(250,200,250))\ncv2.imshow(\'original\',Frames[100])\ncv2.imshow(\'changed\',Nframe)\ncv2.waitKey(0)\ncv2.destroyAllWindows()'

# Add Occlusions

In [28]:
def occlusions(image, num_occlusions):    
    output_image = np.copy(image)
    
    height, width = image.shape[:2]
    
    # Draw random occlusions on the image
    for _ in range(num_occlusions):
        shape_type = random.choice(['rectangle', 'circle'])
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))  # Random color
        
        if shape_type == 'rectangle':
            x = random.randint(0, width - 1)
            y = random.randint(0, height - 1)
            width_rect = random.randint(10, 100)
            height_rect = random.randint(10, 100)
            cv2.rectangle(output_image, (x, y), (x + width_rect, y + height_rect), color, -1)  # Filled rectangle
        
        elif shape_type == 'circle':
            center = (random.randint(0, width - 1), random.randint(0, height - 1))
            radius = random.randint(5, 50)
            cv2.circle(output_image, center, radius, color, -1)  # Filled circle
    
    return output_image

In [29]:
"""New_img = occlusions(Frames[100], 1)

cv2.imshow('Original Image', Frames[100])
cv2.imshow('Image with Occlusions', cv2.cvtColor(New_img,cv2.COLOR_BGR2RGB))
cv2.imshow('Image with Occlusions1', New_img)
cv2.waitKey(0)
cv2.destroyAllWindows()"""

"New_img = occlusions(Frames[100], 1)\n\ncv2.imshow('Original Image', Frames[100])\ncv2.imshow('Image with Occlusions', cv2.cvtColor(New_img,cv2.COLOR_BGR2RGB))\ncv2.imshow('Image with Occlusions1', New_img)\ncv2.waitKey(0)\ncv2.destroyAllWindows()"

# Display Image

In [30]:
def display_frame(I):
    cv2.imshow('I', I)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Read Images in a Directory

In [31]:
def read_images(directory_path):
    images = []
    # List all files in the directory
    for filename in os.listdir(directory_path):
        # Check if the file has a PNG extension
        if filename.lower().endswith('.png'):
            # Construct full file path
            file_path = os.path.join(directory_path, filename)
            # Read the image using OpenCV
            image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)  # cv2.IMREAD_UNCHANGED to keep the alpha channel if present
            if image is not None:
                images.append(image)
            else:
                print(f"Failed to read image: {file_path}")
    return images

# Draw Random Lines

In [32]:
def rnd_lines(image, num_lines):
    # Get the dimensions of the image
    height, width = image.shape[:2]

    # Copy the image to avoid modifying the original
    output_image = image.copy()

    for _ in range(num_lines):
        # Generate random start point
        start_point = (random.randint(0, width-1), random.randint(0, height-1))
        
        # Generate a random angle and length for the line
        angle = random.uniform(0, 2 * np.pi)
        length = random.randint(1, min(width, height) // 2)  # Limit length to half of the smallest dimension
        
        # Calculate the end point using the angle and length
        end_point = (int(start_point[0] + length * np.cos(angle)), 
                     int(start_point[1] + length * np.sin(angle)))
        
        # Ensure the end point is within the image boundaries
        end_point = (min(max(end_point[0], 0), width-1), min(max(end_point[1], 0), height-1))
        
        # Generate a random color (BGR format)
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        
        # Generate a random thickness for the line
        thickness = random.randint(1, 10)
        
        # Draw the line on the image
        cv2.line(output_image, start_point, end_point, color, thickness)

    return output_image

# Random Region's Color Change

In [33]:
def rnd_regions(image, num_regions):
    # Get the dimensions of the image
    height, width = image.shape[:2]

    # Copy the image to avoid modifying the original
    output_image = image.copy()

    for _ in range(num_regions):
        # Generate random region shape and size
        region_shape = random.choice(['rectangle', 'ellipse'])
        
        if region_shape == 'rectangle':
            region_width = random.randint(10, width // 3)
            region_height = random.randint(10, height // 3)
            top_left_x = random.randint(0, width - region_width)
            top_left_y = random.randint(0, height - region_height)
            
            # Define the region
            region = output_image[top_left_y:top_left_y + region_height, top_left_x:top_left_x + region_width]
        
        elif region_shape == 'ellipse':
            center_x = random.randint(width // 3, width - width // 3)
            center_y = random.randint(height // 3, height - height // 3)
            axis_length = (random.randint(10, width // 3), random.randint(10, height // 3))
            angle = random.randint(0, 360)
            start_angle = 0
            end_angle = 360

            # Create a mask for the ellipse
            mask = np.zeros((height, width), dtype=np.uint8)
            cv2.ellipse(mask, (center_x, center_y), axis_length, angle, start_angle, end_angle, 255, -1)
            
            # Extract the region using the mask
            region = cv2.bitwise_and(output_image, output_image, mask=mask)

        # Change color channels within the region
        for channel in range(3):  # Assuming BGR format
            # Generate random ranges for the color channel
            low = random.randint(0, 255)
            high = random.randint(low, 255)
            if region_shape == 'rectangle':
                region[..., channel] = np.clip(region[..., channel], low, high)
            elif region_shape == 'ellipse':
                # Apply changes to the region using the mask
                channel_region = output_image[..., channel]
                channel_region[mask == 255] = np.clip(channel_region[mask == 255], low, high)
                output_image[..., channel] = channel_region

        if region_shape == 'rectangle':
            # Place the modified region back into the image for rectangles
            output_image[top_left_y:top_left_y + region_height, top_left_x:top_left_x + region_width] = region

    return output_image

# Experiments

In [93]:
file = 'VDB\R.mp4'
cap = open_vid(file)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("Width: ",width)
print("Height: ",height)
print("Original FPS: ",original_fps)
print("Frame Count: ",frame_count)
Frames = get_frames(cap)

Width:  640
Height:  360
Original FPS:  24.0
Frame Count:  4755


In [100]:
play_frames(Frames[600:1000],24)

Delay:  41


## With Oclussions

In [79]:
F = Frames.copy()
F[115] = cv2.cvtColor(occlusions(F[115],1),cv2.COLOR_BGR2RGB)
display_frame(F[115])

In [80]:
play_frames(F[100:130],original_fps)

Delay:  40


In [81]:
play_frames(F[100:130],10)

Delay:  100


In [82]:
play_frames(F[100:130],50)

Delay:  20


In [101]:
_ = OFM(Frames[600:700])

In [91]:
_ = OFV(Frames[:300],20)

In [96]:
_ = OFLK(Frames[200:400],20)

In [85]:
_ = PhaseC(F[100:130])

In [98]:
_ = frame_dif(Frames[:300]) 

## With Color Change in Whole Image

In [87]:
F = Frames.copy()
F[115] = change_range_colors(F[115],(0,50,100),(250,200,250))
display_frame(F[115])

In [88]:
play_frames(F[100:130],original_fps)

Delay:  41


In [89]:
play_frames(F[100:130],10)

Delay:  100


In [90]:
play_frames(F[100:130],50)

Delay:  20


In [91]:
_ = OFM(F[100:130])

In [92]:
_ = OFV(F[100:130],20)

In [93]:
_ = OFLK(F[100:130],20)

In [94]:
_ = PhaseC(F[100:130])

In [97]:
_ = frame_dif(Frames[300:400])

## With Color Change in Regions

In [57]:
F = Frames.copy()
F[115] = rnd_regions(F[115],1)
display_frame(F[115])

In [54]:
_ = OFM(F[100:130])

In [55]:
_ = OFV(F[100:130],20)

In [56]:
_ = OFLK(F[100:130],20)

In [57]:
 _ = PhaseC(F[100:130])

In [58]:
_ = frame_dif(F[100:130])

# With Random Lines

In [59]:
F = Frames.copy()
F[115] = rnd_lines(F[115],5)
display_frame(F[115])

In [60]:
_ = OFM(F[100:130])

In [61]:
_ = OFV(F[100:130],20)

In [62]:
_ = OFLK(F[100:130],5)

In [63]:
 _ = PhaseC(F[100:130])

In [64]:
_ = frame_dif(F[100:130])

# Manual Alterations

In [96]:
#delete_png_files('saved_frames')
#save_frames(Frames[100:130],"saved_frames")

In [63]:
K = read_images('saved_frames')

In [66]:
play_frames(K,24)

Delay:  41


In [67]:
print(K[15].shape)
K[15] = K[15][:,:,:3]
print(K[15].shape)

(360, 640, 4)
(360, 640, 3)


In [68]:
_ = OFM(K)

In [69]:
_ = OFV(K,20)

In [70]:
_ = OFLK(K,20)

In [103]:
 _ = PhaseC(K)

In [104]:
_ = frame_dif(K)

In [ ]:
#Suma de vectores
#Usar flechas para ir hacia adelante y atras
#CHecar moviemiento de camara en vectores
#calcular norma de suma de vectores
#Cuales excenas son de acercamiento o alejamiento o que tipo de movimiento de camara
#ruido en frames consecutivos
#Separar en cada canal de color
#Checar movimiento de siluetas
#Manipulacion de camara, dif movimientos (horizontal,vertical,etc)